In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요 라이브러리 설치

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.4 MB/s 
     |████████████████████████████████| 7.6 MB 67.3 MB/s 
     |████████████████████████████████| 182 kB 99.8 MB/s 


# KoGPT2 model, tokenizer 호출

In [3]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel
import torch


model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
# GPU 사용 여부
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model.to(device)
model.eval()
print(device)

cuda:0


In [ ]:
# 토큰과, 예측하고 싶은 문장 개수를 k로 입력
# 토큰으로 시작하는 문장 k개 반환
def infer_sentence(input_sentence, k, output_token_length):

    # 입력 받은 문장 인코딩
    input_ids = tokenizer.encode(
        input_sentence, 
        add_special_tokens=False, 
        return_tensors="pt"
        )

    # 결과물을 담을 리스트
    list_decoded_sequences = []

    # 결과물 개수가 k 와 같아질 때까지 반복
    while len(list_decoded_sequences) < k:
        # 인풋 시퀀스로 generate
        output_sequences = model.generate(
            input_ids=input_ids.to(device), 
            do_sample=True, 
            max_length=output_token_length, 
            num_return_sequences=k
            )

        for index, generated_sequence in enumerate(output_sequences):
            # 문장(예측 데이터) 생성
            generated_sequence = generated_sequence.tolist()

            # padding index 앞까지 slicing 함으로써 padding 제거
            generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]

            # 결과물 디코딩
            decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

            # 결과물 리스트에 담기
            list_decoded_sequences.append(decoded_sequence)
        
        # 결과물 중복 제거
        list_decoded_sequences = list(set(list_decoded_sequences))
    
    return list_decoded_sequences

input_sentence = "겨울"
print(f"Inferred sentences given '{input_sentence}'")
inferred_sentences = infer_sentence(input_sentence, k=5, output_token_length=42)
inferred_sentences

Inferred sentences given '겨울'


['겨울은 왜 이렇게 따뜻하고 따뜻했는지',
 '겨울과 같은 시간에',
 '겨울보다 따뜻한 비가 오길',
 '겨울엔 니가 보고싶은 밤',
 '겨울 같지 않은 말']

In [ ]:
# 삼행시를 만들어주는 함수
def make_samhaengshi(input_letter, k, output_token_length):
    # 결과물 list
    list_samhaengshi = []

    for one_letter in input_letter:
        # 한 글자씩 k개의 문장 generate
        list_decoded_sequences = infer_sentence(one_letter, k=k, output_token_length=output_token_length)
        # list에 담긴 k개의 문장을 결과물 list에 extend
        list_samhaengshi.extend(list_decoded_sequences)

    # 결과물 list 반환
    return list_samhaengshi

make_samhaengshi(input_letter="해파리", k=1, output_token_length=42)

['해줄걸 그랬어', '파파와 파도의 나비', '리마득 또']

In [ ]:
def make_samhaengshi(input_letter, k, output_token_length):
    list_samhaengshi = []
    for one_letter in input_letter:
        list_decoded_sequences = infer_sentence(one_letter, k=k, output_token_length=output_token_length)
        list_samhaengshi.extend(list_decoded_sequences)
    return list_samhaengshi

# make_samhaengshi(input_letter="해파리", k=1, output_token_length=10)

# 기존 삼행시 함수

In [ ]:
def make_residual_samhaengshi(input_letter, k, output_token_length):
    # 삼행시 리스트
    list_samhaengshi = []
    
    # initializing text and index for iteration purpose
    index = 0

    # iterating over the input letter string
    for index, letter_item in enumerate(input_letter):
        # initializing the input_letter
        if index == 0:
            residual_text = letter_item
        else:
            pass
        
        # infer and add to the output
        list_sentences = infer_sentence(residual_text, 3, output_token_length)
        for sentence in list_sentences:
            if len(sentence) == 1:
                pass
            elif len(sentence) >= 2:
                inferred_sentence = sentence # first item of the inferred list
        if index != 0:
            # remove previous sentence from the output
            inferred_sentence = inferred_sentence.replace(list_samhaengshi[index-1], "").strip() 
        else:
            pass
        list_samhaengshi.append(inferred_sentence)
        
        # until the end of the input_letter, give the previous residual_text to the next iteration
        if index < len(input_letter) - 1: 
            residual_sentence = list_samhaengshi[index]
            next_letter = input_letter[index + 1]
            residual_text = f"{residual_sentence} {next_letter}" #  previous sentence + next letter
            # print(residual_text)

        elif index == len(input_letter) - 1: # end of the input_letter
            # Concatenate strings in the list without intersection

            return list_samhaengshi

In [ ]:
sample_item = "박정우" 
inferred_samhaengshi = make_residual_samhaengshi(sample_item, k=1, output_token_length=42)
for item in inferred_samhaengshi:
    print(item)

박필란 박근히 조각내
정말로 너인 걸 알기에 난 너인 걸
우린데


In [ ]:
sample_item = "멋쟁이" 
inferred_samhaengshi = make_residual_samhaengshi(sample_item, k=1, output_token_length=42)
for item in inferred_samhaengshi:
    print(item)

멋지게 하지마
쟁기굽대
이 밤들이


In [ ]:
sample_item = "파이널" 
inferred_samhaengshi = make_residual_samhaengshi(sample_item, k=1, output_token_length=42)
for item in inferred_samhaengshi:
    print(item)

파란한 마음에 취해버리면
이 길 끝은 보이지 않으니
널


In [ ]:
sample_item = "해파리" 
inferred_samhaengshi = make_residual_samhaengshi(sample_item, k=1, output_token_length=42)
for item in inferred_samhaengshi:
    print(item)

해진 않았다
파도의 마왕처럼
리오


# 파인 튜닝 데이터

In [5]:
import pandas as pd
import numpy as np

In [6]:
f = open("/content/drive/MyDrive/Colab Notebooks/data/ballad_all.txt", 'r')
lines = f.readlines()
f.close()

In [7]:
# 문장 개수, 전체 발라드 약 56,000곡
len(lines)

966651

# tokenizer

In [27]:
# 문장 50,000개만 샘플링해서 일단 돌려보자
man_lines = list(np.random.choice(lines, 50000))
removed_lines = [line.replace('\n', '') for line in man_lines]
list(np.random.choice(removed_lines, 10))

['그녀는 신경 쓰지 마 나만 몰랐었어 믿고 싶지 않아',
 '사랑합니다 날 불러준 그대',
 '오늘도 난 너만 바라보는데',
 '꿈보다 더 꿈같은',
 '그대를 기억하고',
 '아무도 모르는 눈물 초라한 날보며 웃어도',
 '그 사랑을 담아',
 '오늘도 난 바래',
 '보이지 않나요 나',
 '그대 옆의 그녀를 바라보아요']

In [9]:
# removed_lines = [line.replace('\n', '') for line in lines]
# list(np.random.choice(removed_lines, 10))

['잘해주지 못한 기억만 자꾸 떠올라',
 '이렇게 널 이제 보내줘야 하나',
 '노을만 붉게 타는데',
 '꿇겠다고 내 사랑아',
 '반지 낀 손가락 사이로 눈물이 뚝뚝 떨어지는데 울지 말란 말야 이 바보야 뭐라고 말을 해 말해줘',
 '너를 대할때마다',
 '사진속 그대는',
 '그렇게 그녈 떠나보냈던 건',
 '눈 앞에 찬비가 넘치듯 쏟아 내려도',
 '행복했던 그 순간들을']

In [28]:
len(removed_lines)

50000

In [29]:
tokenized_datasets = tokenizer(removed_lines, 
                               return_tensors="pt", 
                               padding="max_length", 
                               max_length=42,
                               truncation=True)

In [30]:
tokenized_datasets

{'input_ids': tensor([[ 9669,  7055,  9134,  ...,     3,     3,     3],
        [ 9022,  9068,  9022,  ...,     3,     3,     3],
        [ 9022, 42076, 17270,  ...,     3,     3,     3],
        ...,
        [17582,  9019,  8162,  ...,     3,     3,     3],
        [ 9114,  9114,     3,  ...,     3,     3,     3],
        [11781, 10224, 11318,  ...,     3,     3,     3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [31]:
len(tokenized_datasets["input_ids"])

50000

In [32]:
# https://huggingface.co/transformers/custom_datasets.html
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    """ CustomDataset class for poetic sentences """
    def __init__(self, list_dataset, tokenizer):

        self.list_dataset = list_dataset
        self.tokenizer = tokenizer
        self.tokenized_sentences = self.tokenizer(
            list_dataset,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=42,
            add_special_tokens=True,
            return_token_type_ids=False,
            )

    def __getitem__(self, idx):
        encoded_dict = {key: val[idx] for key, val in self.tokenized_sentences.items()}
        encoded_dict["labels"] = encoded_dict["input_ids"].clone() # gpt has same labels as input_ids: https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb
        return encoded_dict

    def __len__(self):
        return len(self.list_dataset)

In [33]:
tokenized_datasets = CustomDataset(removed_lines, tokenizer)
# kor_eval_dataset_for_debugging = CustomDataset(removed_lines[:100], tokenizer) # small dataset for debugging purpose
print(len(tokenized_datasets))

50000


# 파인 튜닝

In [34]:
from transformers import TrainingArguments

# Trainer가 학습, 평가에 사용할 모든 하이퍼 파라미터를 포함하는 클래스 정의
# 학습된 모델이 저장될 디렉토리만 지정하고 나머지는 기본값 사용
training_args = TrainingArguments(
    "/content/drive/MyDrive/Colab Notebooks/ballad_all_model",      
    logging_steps=10000,
    warmup_steps=10000,
    save_steps=10000,
    eval_steps=10000)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [35]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

In [36]:
trainer.train()

***** Running training *****
  Num examples = 50000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18750
  Number of trainable parameters = 125164032


Step,Training Loss
10000,0.657000


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-10000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-10000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-10000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-10000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18750, training_loss=0.6074532291666667, metrics={'train_runtime': 1071.7283, 'train_samples_per_second': 139.961, 'train_steps_per_second': 17.495, 'total_flos': 3215116800000000.0, 'train_loss': 0.6074532291666667, 'epoch': 3.0})

# 파인 튜닝 후 내 맘대로 삼행시

In [37]:
def mind(input_letter):
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
 
        # 만약 idx 가 0 이라면 == 첫 글자
        if idx == 0:
            # 첫 글자 인코딩
            input_ids = tokenizer.encode(
            val, add_special_tokens=False, return_tensors="pt")
            
            # 첫 글자 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids.to(device), 
                do_sample=True, max_length=42)
        
        # 첫 글자가 아니라면
        else:
            # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
            # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
            link_with_pre_sentence = " ".join(res_l) + " " + val  
            # print(link_with_pre_sentence)

            # 연결된 문장을 인코딩
            input_ids = tokenizer.encode(
            link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

            # 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids.to(device), 
                do_sample=True, max_length=42)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]

        # padding index 앞까지 slicing 함으로써 padding 제거
        generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
        
        # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
        # print(generated_sequence)
        if idx != 0:
            # 이전 문장의 마지막 시퀀스 이후로 슬라이싱해서 앞 문장 제거
            generated_sequence = generated_sequence[generated_sequence.index(last_sequence) + 1:]

            # 다음 음절을 위해 마지막 시퀀스 갱신
            last_sequence = generated_sequence[-1]        
        
        # 첫 글자라면
        else:
            # 마지막 시퀀스 저장
            last_sequence = generated_sequence[-1]        
        
        # print(last_sequence)

        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

        # print(res_l)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)

In [73]:
mind("박정우")

박 : 박박혀서 손안에 차올라
정 : 정말로 아름답기도 했었지
우 : 우릴 떠나려는지 아니었는데 시간이 가면 모든 것들을 잊었지 우린 추억이 우릴 물들이고


In [71]:
mind("해파리")

해 : 해주고 싶은 게 뭐냐 말야
파 : 파랗게 빛나는 별이 예뻐
리 : 리본 나의 맘이 너를 만나던 그 날


In [46]:
mind("파이널")

파 : 파랗게 쌓인 날 보네
이 : 이젠 모두 끝인 거야
널 : 널 위한


In [41]:
mind("멋쟁이")

멋 : 멋나게 사는 나지만
쟁 : 쟁세라서
이 : 이 밤이 지나도
